# To-do list

- convert stringency data to float
- download and process additional data from mzcr ? 
- merge the datasets

In [207]:
#importing required packages 
import requests
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from bs4 import BeautifulSoup

###### Processing the stringency data by json subscription to be then converting them to data frame

In [119]:
# API to obtain the stringency data https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/{YYYY-MM-DD}/{YYYY-MM-DD}

start_date= '2020-03-10'
end_date= '2020-06-20'

r = requests.get(f'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/{start_date}/{end_date}')

In [121]:
dt = r.json()

In [122]:
data = {}
for i in dt['data']:
    for j in dt['data'][i]:
        if dt['data'][i][j]['country_code'] == 'CZE':
            date = dt['data'][i][j]['date_value']
            data[date] = dt['data'][i][j]
            continue

In [123]:
df = pd.DataFrame(data).T[['confirmed', 'deaths',
       'stringency_actual', 'stringency', 'stringency_legacy',
       'stringency_legacy_disp']]
df

,confirmed,deaths,stringency_actual,stringency,stringency_legacy,stringency_legacy_disp
2020-03-10,41,0,25,25,33.57,33.57
2020-03-11,91,0,47.22,47.22,53.57,53.57
2020-03-12,94,0,50,50,53.57,53.57
2020-03-13,141,0,57.41,57.41,64.29,64.29
2020-03-14,189,0,61.11,61.11,67.86,67.86
...,...,...,...,...,...,...
2020-06-16,10111,331,41.67,41.67,52.38,52.38
2020-06-17,10162,333,41.67,41.67,52.38,52.38
2020-06-18,10280,334,41.67,41.67,52.38,52.38
2020-06-19,10406,335,41.67,41.67,52.38,52.38


#### Scrapping hospitalization data from https://onemocneni-aktualne.mzcr.cz/covid-19

In [133]:
r = requests.get('https://onemocneni-aktualne.mzcr.cz/covid-19')

In [165]:
soup = BeautifulSoup(r.text,'lxml')

In [205]:
a = json.loads(soup.find('div', {'class':'visually-hidden', 'id' : 'js-hospitalization-table-data'})['data-table'])


In [206]:
df_hosp = pd.DataFrame(a['body'], columns = ['Date','Akt. hosp', 'V těžkém stavu', '% v těžkém stavu']).set_index('Date')
df_hosp

,Akt. hosp,V těžkém stavu,% v těžkém stavu
Date,,,
01.03.2020,0,0,0.0000
02.03.2020,0,0,0.0000
03.03.2020,0,0,0.0000
04.03.2020,0,0,0.0000
05.03.2020,0,0,0.0000
...,...,...,...
10.05.2021,1981,369,0.1863
11.05.2021,1829,351,0.1919
12.05.2021,1711,328,0.1917


In [208]:
df_hosp.describe()

,Akt. hosp,V těžkém stavu,% v těžkém stavu
count,440.000000,440.000000,440.000000
mean,2989.345455,538.018182,0.182313
std,3088.010045,587.404650,0.052631
min,0.000000,0.000000,0.000000
25%,113.000000,24.000000,0.150000
50%,1555.500000,315.500000,0.189800
75%,5965.500000,1023.000000,0.217800
max,9461.000000,2062.000000,0.309100


### not usefull anymore - to be deleted
##### Trying to process the data as data frame 

In [36]:
pd.DataFrame(dt['data'])

,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20
AFG,"{'date_value': '2020-03-10', 'country_code': '...","{'date_value': '2020-03-11', 'country_code': '...","{'date_value': '2020-03-12', 'country_code': '...","{'date_value': '2020-03-13', 'country_code': '...","{'date_value': '2020-03-14', 'country_code': '...","{'date_value': '2020-03-15', 'country_code': '...","{'date_value': '2020-03-16', 'country_code': '...","{'date_value': '2020-03-17', 'country_code': '...","{'date_value': '2020-03-18', 'country_code': '...","{'date_value': '2020-03-19', 'country_code': '...","{'date_value': '2020-03-20', 'country_code': '..."
ALB,"{'date_value': '2020-03-10', 'country_code': '...","{'date_value': '2020-03-11', 'country_code': '...","{'date_value': '2020-03-12', 'country_code': '...","{'date_value': '2020-03-13', 'country_code': '...","{'date_value': '2020-03-14', 'country_code': '...","{'date_value': '2020-03-15', 'country_code': '...","{'date_value': '2020-03-16', 'country_code': '...","{'date_value': '2020-03-17', 'country_code': '...","{'date_value': '2020-03-18', 'country_code': '...","{'date_value': '2020-03-19', 'country_code': '...","{'date_value': '2020-03-20', 'country_code': '..."
DZA,"{'date_value': '2020-03-10', 'country_code': '...","{'date_value': '2020-03-11', 'country_code': '...","{'date_value': '2020-03-12', 'country_code': '...","{'date_value': '2020-03-13', 'country_code': '...","{'date_value': '2020-03-14', 'country_code': '...","{'date_value': '2020-03-15', 'country_code': '...","{'date_value': '2020-03-16', 'country_code': '...","{'date_value': '2020-03-17', 'country_code': '...","{'date_value': '2020-03-18', 'country_code': '...","{'date_value': '2020-03-19', 'country_code': '...","{'date_value': '2020-03-20', 'country_code': '..."
AND,"{'date_value': '2020-03-10', 'country_code': '...","{'date_value': '2020-03-11', 'country_code': '...","{'date_value': '2020-03-12', 'country_code': '...","{'date_value': '2020-03-13', 'country_code': '...","{'date_value': '2020-03-14', 'country_code': '...","{'date_value': '2020-03-15', 'country_code': '...","{'date_value': '2020-03-16', 'country_code': '...","{'date_value': '2020-03-17', 'country_code': '...","{'date_value': '2020-03-18', 'country_code': '...","{'date_value': '2020-03-19', 'country_code': '...","{'date_value': '2020-03-20', 'country_code': '..."
AGO,"{'date_value': '2020-03-10', 'country_code': '...","{'date_value': '2020-03-11', 'country_code': '...","{'date_value': '2020-03-12', 'country_code': '...","{'date_value': '2020-03-13', 'country_code': '...","{'date_value': '2020-03-14', 'country_code': '...","{'date_value': '2020-03-15', 'country_code': '...","{'date_value': '2020-03-16', 'country_code': '...","{'date_value': '2020-03-17', 'country_code': '...","{'date_value': '2020-03-18', 'country_code': '...","{'date_value': '2020-03-19', 'country_code': '...","{'date_value': '2020-03-20', 'country_code': '..."
...,...,...,...,...,...,...,...,...,...,...,...
ZMB,"{'date_value': '2020-03-10', 'country_code': '...","{'date_value': '2020-03-11', 'country_code': '...","{'date_value': '2020-03-12', 'country_code': '...","{'date_value': '2020-03-13', 'country_code': '...","{'date_value': '2020-03-14', 'country_code': '...","{'date_value': '2020-03-15', 'country_code': '...","{'date_value': '2020-03-16', 'country_code': '...","{'date_value': '2020-03-17', 'country_code': '...","{'date_value': '2020-03-18', 'country_code': '...","{'date_value': '2020-03-19', 'country_code': '...","{'date_value': '2020-03-20', 'country_code': '..."
ZWE,"{'date_value': '2020-03-10', 'country_code': '...","{'date_value': '2020-03-11', 'country_code': '...","{'date_value': '2020-03-12', 'country_code': '...","{'date_value': '2020-03-13', 'country_code': '...","{'date_value': '2020-03-14', 'country_code': '...","{'date_value': '2020-03-15', 'country_code': '...","{'date_value': '2020-03-16', 'country_code': '...","{'dat

In [84]:
df= pd.DataFrame(dt['data'])
df[df.index == 'CZE'].T

,CZE
2020-03-10,"{'date_value': '2020-03-10', 'country_code': '..."
2020-03-11,"{'date_value': '2020-03-11', 'country_code': '..."
2020-03-12,"{'date_value': '2020-03-12', 'country_code': '..."
2020-03-13,"{'date_value': '2020-03-13', 'country_code': '..."
2020-03-14,"{'date_value': '2020-03-14', 'country_code': '..."
2020-03-15,"{'date_value': '2020-03-15', 'country_code': '..."
2020-03-16,"{'date_value': '2020-03-16', 'country_code': '..."
2020-03-17,"{'date_value': '2020-03-17', 'country_code': '..."
2020-03-18,"{'date_value': '2020-03-18', 'country_code': '..."
2020-03-19,"{'date_value': '2020-03-19', 'country_code': '..."
